In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

df1 = pd.read_csv('Data_train.csv')
df1

,ACCT_NMB,RECENCY_DAY,FREQUENCY,MONETORY,RMF_SCORE,DOTCOM,REWARDS_ACCOUNT,FREQ_1,FREQ_2,FREQ_3,...,FREQ_11,FREQ_12,CNT_PER_PDT,CNT_PER_PDT_SFC,CNT_PER_PDT_VFC,NO_DISCOUNT,DISCOUNT_PROMOTION,SUM_DIS_PRO,label,features
0,8888,11,25,377.11,3.000000,0,1,8,0,7,...,9,8,0,0,0,106,6,-33.42,0,"{""vectorType"":""dense"",""length"":24,""values"":[11..."
1,10710,74,9,54.48,1.750000,0,1,5,7,0,...,0,5,0,0,0,113,4,-17.71,1,"{""vectorType"":""dense"",""length"":24,""values"":[74..."
2,11031,17,12,177.53,2.583333,0,1,11,0,1,...,1,2,0,0,0,20,0,0.00,0,"{""vectorType"":""sparse"",""length"":24,""indices"":[..."
3,18903,173,1,13.99,0.500000,0,1,0,0,0,...,6,0,0,0,0,7,0,0.00,0,"{""vectorType"":""sparse"",""length"":24,""indices"":[..."
4,18986,169,5,115.50,1.500000,1,1,2,8,7,...,6,1,0,0,0,33,5,-22.61,0,"{""vectorType"":""dense"",""length"":24,""values"":[16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10560753,46,14,225.76,2.500000,0,1,11,0,0,...,9,0,0,0,0,50,1,-26.99,0,"{""vectorType"":""dense"",""length"":24,""values"":[46..."
996,10564680,6,75,1466.31,3.000000,0,1,12,20,18,...,19,17,52,3,7,197,2,-14.97,1,"{""vectorType"":""dense"",""length"":24,""values"":[6,..."
997,10569820,60,89,1447.32,2.666667,1,1,37,11,15,...,13,20,64,2,3,328,16,-102.40,1,"{""vectorType"":""dense"",""length"":24,""values"":[60..."
998,10570422,77,11,689.68,2.416667,0,1,11,6,5,...,0,4,0,0,0,63,1,-10.00,0,"{""vectorType"":""dense"",""length"":24,""values"":[77..."


In [2]:
features=["RECENCY_DAY",
"FREQUENCY",
"MONETORY",
"RMF_SCORE",
"DOTCOM",
"REWARDS_ACCOUNT",
"FREQ_1",
"FREQ_2",
"FREQ_3",
"FREQ_4",
"FREQ_5",
"FREQ_6",
"FREQ_7",
"FREQ_8",
"FREQ_9",
"FREQ_10",
"FREQ_11",
"FREQ_12",
"CNT_PER_PDT",
"CNT_PER_PDT_SFC",
"CNT_PER_PDT_VFC",
"NO_DISCOUNT",
"DISCOUNT_PROMOTION",
"SUM_DIS_PRO"]

In [3]:
X = df1[features].values
y = df1['label'].values

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [5]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 10, verbose=1,factor=0.3, min_lr=0.0000001)

In [6]:
early_stopping = EarlyStopping()
custom_early_stopping = [
    EarlyStopping(
    monitor='val_accuracy', 
    patience=20, 
    min_delta=0.0001, 
    mode='auto'
)]

In [7]:
def twoLayerFeedForward():
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization

    clf = Sequential()
    # clf.add(Dense(2, activation='relu', input_dim=24))
    # clf.add(Dense(2, activation='relu'))
    # clf.add(Dense(2, activation='sigmoid'))
    clf.add(Dense(units = 128 , activation = 'relu'))
    clf.add(Dropout(0.2))
    clf.add(Dense(units = 1 , activation = 'sigmoid')) # Tanh
    clf.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=["accuracy"])
    return clf

In [8]:
def build_tf_model(p_df: pd.DataFrame):
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    import tensorflow as tf
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
    from tensorflow.keras import datasets, layers, models
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.callbacks import ReduceLROnPlateau
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
    numeric_features = p_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = p_df.select_dtypes(include=['object']).columns.tolist()
    
    feature_names = numeric_features + categorical_features

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler(with_mean=True,with_std=True))])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    model = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier'
                    ,KerasClassifier(twoLayerFeedForward)
                )
            ])

    return model

In [9]:
modelpipe = build_tf_model(pd.DataFrame(X_train))

In [10]:
modelpipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('classifier',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f883d5bec40>)])

In [11]:
tf = modelpipe.fit(X_train,y_train, classifier__epochs=200, classifier__batch_size=500, classifier__verbose=1,classifier__validation_data = (X_test, y_test),classifier__callbacks=[custom_early_stopping])

2022-12-21 18:22:50.263607: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 18:22:50.303079: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/200
2/2 [==============================] - 1s 382ms/step - loss: 0.8527 - accuracy: 0.4240 - val_loss: 12.8892 - val_accuracy: 0.6100
Epoch 2/200
2/2 [==============================] - 0s 21ms/step - loss: 0.6742 - accuracy: 0.6212 - val_loss: 15.2640 - val_accuracy: 0.6050
Epoch 3/200
2/2 [==============================] - 0s 22ms/step - loss: 0.5997 - accuracy: 0.7022 - val_loss: 16.1338 - val_accuracy: 0.5950
Epoch 4/200
2/2 [==============================] - 0s 21ms/step - loss: 0.5675 - accuracy: 0.7235 - val_loss: 15.8684 - val_accuracy: 0.6000
Epoch 5/200
2/2 [==============================] - 0s 22ms/step - loss: 0.5503 - accuracy: 0.7350 - val_loss: 16.1981 - val_accuracy: 0.6000
Epoch 6/200
2/2 [==============================] - 0s 22ms/step - loss: 0.5248 - accuracy: 0.7517 - val_loss: 15.0127 - val_accuracy: 0.6000
Epoch 7/200
2/2 [==============================] - 0s 22ms/step - loss: 0.5257 - accuracy: 0.7508 - val_loss: 14.5201 - val_accuracy: 0.6150
Epoch 8/200


In [12]:
def get_classification_report(y_test, y_pred):
    from sklearn import metrics
    report = metrics.classification_report(y_test, y_pred, output_dict=True,target_names=['0','1'])
    df_classification_report = pd.DataFrame(report).transpose()    
    return df_classification_report

In [13]:
get_classification_report(tf.predict(X_test),y_test).reset_index().rename(columns={"index": "class"}).reset_index(drop=True)

/opt/anaconda3/envs/snowpark_012/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


,class,precision,recall,f1-score,support
0,0,0.920863,0.847682,0.882759,151.00
1,1,0.622951,0.775510,0.690909,49.00
2,accuracy,0.830000,0.830000,0.830000,0.83
3,macro avg,0.771907,0.811596,0.786834,200.00
4,weighted avg,0.847875,0.830000,0.835755,200.00


In [14]:
tf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 19, 20, 21, 22,
                                                   23]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('classifier',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f883d5bec40>)])

In [15]:
# collect the preprocessing pipeline & model seperately
import joblib

# Save the Keras model first:
tf.named_steps['classifier'].model.save('kmodel.h5')

# This hack allows us to save the sklearn pipeline:
tf.named_steps['classifier'].model = None

# Finally, save the pipeline:
joblib.dump(tf, 'sklearn_pipe.pkl')

['sklearn_pipe.pkl']

In [17]:
# Load the pipeline first:
from tensorflow.keras.models import load_model
import joblib
pipeline = joblib.load('sklearn_pipe.pkl')

# Then, load the Keras model:
tf.named_steps['classifier'].model = load_model('kmodel.h5')

y_pred = tf.predict(X_test)
y_pred

/opt/anaconda3/envs/snowpark_012/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
    